'''
For the below dataset :
|Product|  Store|Price|
+-------+-------+-----+
| Iphone|BestBuy|  100|
| Iphone|Walmart|   90|
| Iphone| Amazon|   95|
|Samsung| Amazon|   80|
|Samsung|Walmart|   85|
|Samsung|BestBuy|   90|

Write a Query to generate the output as below without using window functions:
|Product|  Walmart_price|BestBuy_price|Amazon_price|Competitive_price|
+-------+-------+-----+
| Iphone|90|  100|  95| N
| Samsung|85|  90|   80| Y
'''

In [32]:
#Using Window Function
import os, sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder.master("local[*]").appName("Competitive_Price").getOrCreate()

class AmazonInventory:
    
    def get_data(self):
        columns = ['Product','Store','Price']
        data = [
            ('Iphone','BestBuy',100),
            ('Iphone','Walmart',90),
            ('Iphone','Amazon',95),
            ('Samsung','BestBuy',90),
            ('Samsung','Walmart',85),
            ('Samsung','Amazon',80)
        ]
        inputDf = spark.createDataFrame(data, columns) 
        return inputDf
    
    def priceCheck(self, inputDf):
        inputDf.createOrReplaceTempView('input')
        
        query = """
        with pivot_cte as (
        select product, 
        sum(case when Store = 'Walmart' then Price else 0 end) as Walmart_price,
        sum(case when Store = 'BestBuy' then Price else 0 end) as BestBuy_price,
        sum(case when Store = 'Amazon' then Price else 0 end) as Amazon_price
        from input
        group by product)
        
        select product, Walmart_price, BestBuy_price, Amazon_price, case when A.rnk=1 then 'Y' else 'N' end as Competitive_price
        from (
        select product,Walmart_price, BestBuy_price, Amazon_price, dense_rank() over(order by Amazon_price) as rnk
        from pivot_cte)A 
        """
        return spark.sql(query)
    
    def priceCheckPySpark(self, inputDf):
        pivotDf = inputDf.groupBy('Product')\
        .agg(sum(when(col('Store')=='Walmart', col("Price")).otherwise(0)).alias("Walmart_price"),
                 sum(when(col('Store')=='BestBuy', col("Price")).otherwise(0)).alias("BestBuy_price"),
                 sum(when(col('Store')=='Amazon', col("Price")).otherwise(0)).alias("Amazon_price")
                )
        
        # Rank the prices by Amazon_price
        rank_df = pivotDf.withColumn("rnk", dense_rank().over(Window.orderBy(col("Amazon_price"))))
        
        # Select the required columns and calculate Competitive_price
        result_df = rank_df.select('Product', 'Walmart_price','BestBuy_price', 'Amazon_price', when(col('rnk')==1,'Y').otherwise('N').alias('Competitive_price'))
        
        return result_df
    
obj = AmazonInventory()
inputDf = obj.get_data()
inputDf.show()
outputDf = obj.priceCheck(inputDf)
outputDf.show()
outputDf = obj.priceCheckPySpark(inputDf)
outputDf.show()

+-------+-------+-----+
|Product|  Store|Price|
+-------+-------+-----+
| Iphone|BestBuy|  100|
| Iphone|Walmart|   90|
| Iphone| Amazon|   95|
|Samsung|BestBuy|   90|
|Samsung|Walmart|   85|
|Samsung| Amazon|   80|
+-------+-------+-----+

+-------+-------------+-------------+------------+-----------------+
|product|Walmart_price|BestBuy_price|Amazon_price|Competitive_price|
+-------+-------------+-------------+------------+-----------------+
|Samsung|           85|           90|          80|                Y|
| Iphone|           90|          100|          95|                N|
+-------+-------------+-------------+------------+-----------------+

+-------+-------------+-------------+------------+-----------------+
|Product|Walmart_price|BestBuy_price|Amazon_price|Competitive_price|
+-------+-------------+-------------+------------+-----------------+
|Samsung|           85|           90|          80|                Y|
| Iphone|           90|          100|          95|                N|

In [33]:
import os, sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark = SparkSession.builder.master('local[*]').getOrCreate()
class AmazonInventory:
    def createData(self):
        data = [
            ('Iphone', 'BestBuy', 100),
            ('Iphone', 'Walmart', 90),
            ('Iphone', 'Amazon', 95),
            ('Samsung', 'Amazon', 80),
            ('Samsung', 'Walmart', 85),
            ('Samsung', 'BestBuy', 90)
        ]
        schema = ['Product', 'Store', 'Price']
        return spark.createDataFrame(data, schema)

    #SQL approach
    def priceCheck(self, inputDf):
        inputDf.createOrReplaceTempView('input')
        query = """with pivot as (
                select * from input pivot(
                max(Price) for Store in ('Walmart' as Walmart_price, 'BestBuy' as BestBuy_price, 'Amazon' as Amazon_price)
                ) ),
                minPrice as (
                select min(Price) as min_price, Product from input group by Product
                )
                select M1.*,
                case when M1.Amazon_price = M2.min_price
                    then 'Y'
                    else 'N'
                end as Competitive_price
                from pivot M1 inner join minPrice M2
                on M1.Product = M2.Product
                order by M1.Product
                """
        return spark.sql(query)

    #PySpark approach
    def amazonPriceCheck(self, inputDf):
        minPricedf = inputDf.groupBy('Product').agg(min('Price').alias('Min_Price'))
        pivotDf = inputDf.groupBy('Product').pivot('Store').max('Price')
        resultDf = pivotDf.join(minPricedf, on='Product', how='inner')\
                        .withColumn('Competitive_price', when(col('Amazon').__eq__(col('Min_Price')), lit('Y')).otherwise(lit('N')))\
                        .select('Product', col('BestBuy').alias('BestBuy_Price'), col('Walmart').alias('Walmart_Price'), col('Amazon').alias('Amazon_Price'), 'Competitive_price')\
                        .orderBy('Product')
        return resultDf

ob = AmazonInventory()
inputDf = ob.createData()

resultDf = ob.priceCheck(inputDf)
resultDf.show()

resultDf = ob.amazonPriceCheck(inputDf)
resultDf.show()

+-------+-------------+-------------+------------+-----------------+
|Product|Walmart_price|BestBuy_price|Amazon_price|Competitive_price|
+-------+-------------+-------------+------------+-----------------+
| Iphone|           90|          100|          95|                N|
|Samsung|           85|           90|          80|                Y|
+-------+-------------+-------------+------------+-----------------+

+-------+-------------+-------------+------------+-----------------+
|Product|BestBuy_Price|Walmart_Price|Amazon_Price|Competitive_price|
+-------+-------------+-------------+------------+-----------------+
| Iphone|          100|           90|          95|                N|
|Samsung|           90|           85|          80|                Y|
+-------+-------------+-------------+------------+-----------------+

